In [91]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [92]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])

print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  15945
Number of unique ingredients:  6899
Number of total ingredients:  446618


In [93]:
df = dataPrep(ings, prods, prod_ing)
#df.to_csv('recommendations/recommender_products.csv')

In [94]:
df = df[df['ingCount'] > 2]  
df_cos = df
print('Number of products: ', len(df))

Number of products:  15848


In [95]:
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]

#remove water
df['ing#List'] = [remove_values_from_list(i,0) for i in df['ing#List']]

In [96]:
df.sort_values(by='ingCount', ascending=True).head(3)

ingList            ing#List  \
13316                     [Water, Propylene Glycol, nan]         [120, 4954]   
5634   [Rosa Canina (Rosehip) Seed Oil, Mentha Piperi...  [1968, 1561, 3648]   
7430   [Petrolatum, Aloe Barbadensis Leaf Extract, Fl...      [24, 317, 793]   

                        product                brand  price  rating  \
13316        Replenishing Cream           Dr. Sebagh    NaN     0.0   
5634              Lip Balm Trio  Rosebud Perfume Co.    NaN     0.0   
7430   Lip Moisturizer Tin Aloe             Vaseline    NaN     0.0   

       ratingsCount  cat_Acne&BlemishTreatments  cat_Ampoules  \
13316             0                           0             0   
5634              0                           0             0   
7430              0                           0             0   

       cat_BodySkincare  ...    note_Brightening  note_ContainsAlcohol  \
13316                 0  ...                   0                     0   
5634                  0  ...                   0                     0   
7430                  0  ...                   0                     0   

       note_ContainsParaben  note_ContainsSulfate  note_ParabenFree  \
13316                     1                     0                 0   
5634                      0                     0                 1   
7430                      0                     0                 1   

       note_PromotesWoundHealing  note_SulfateFree  note_UVProtection  \
13316                          0                 1                  0   
5634                           1                 1                  0   
7430                           0                 1                  0   

       ingCount    id2  
13316         3  16271  
5634          3   6308  
7430          3   8381  

[3 rows x 47 columns]

# GET RECOMMENDATIONS


In [97]:
#id: 1961
product ='C E Ferulic Serum'
ingTest = df[df['product'].str.match(product)]
ingTest = ingTest['ingList']
print(*ingTest, sep='\n')

['Water', 'Ethoxydiglycol', 'Ascorbic Acid (Vitamin C', 'Propylene Glycol', 'Glycerin', 'Laureth-23', 'Tocopherol (Vitamin E', 'Phenoxyethanol', 'Triethanolamine', 'Ferulic Acid', 'Panthenol', 'Sodium Hyaluronate']


## Cosine similarity

Reference [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

In [100]:
%%time

recommended = get_COS_recommendations(df_cos,product)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df_cos)
recommended.to_csv('recommendations/COS.csv')

Wall time: 17.1 s


In [101]:
topTen

product  \
4437                                 Eau Fraiche Douceur   
10285  Eau Frache Douceur Micellar Cleansing Water Fa...   
744                    Micellar Water Cleansing Solution   
15664               Micellar Water 3 In 1 (Moisturizing)   
1370      Purete Thermale 3-In-1 One Step Micellar Water   
7186            Purete Thermale 3 in 1 Micellar Solution   
788    Micellar Cleansing Water Complete Cleanser Nor...   
1975   Nutritioniste SkinActive Micellar Cleansing Wa...   
693    Micellar Cleansing Water Complete Cleanser Nor...   
2204                Pure Active Micellar Water Oily Skin   

                                                 ingList  commonIng  
4437   [Water, Hexylene Glycol, Glycerin, Poloxamer 1...          3  
10285  [Water, Hexylene Glycol, Glycerin, Poloxamer 1...          3  
744    [Water, Hexylene Glycol, Poloxamer 184, Glycer...          2  
15664  [Water, Hexylene Glycol, Glycerin, Rosa Gallic...          3  
1370   [Water, Hexylene Glycol, Glycerin, Poloxamer 1...          3  
7186   [Water, Hexylene Glycol, Glycerin, Poloxamer 1...          3  
788    [Water, Hexylene Glycol, Glycerin, Poloxamer 1...          2  
1975   [Water, Hexylene Glycol, Glycerin, Disodium Co...          2  
693    [Water, Hexylene Glycol, Poloxamer 184, Disodi...          1  
2204   [Water, Hexylene Glycol, Disodium Cocoamphodia...          1

## Average Overlap

In [76]:
#df['ing#List'] = [ast.literal_eval(i) for i in df['ing#List']]
indices = pd.Series(df.index, index=df['product']).drop_duplicates()
idx = indices[product]

itemLookup = df.loc[idx]['ing#List']
items = df['ing#List']
sim = [average_overlap(itemLookup,i) for i in items] ##change method here
sim_scores = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores = sim_scores[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/AO.csv')

In [77]:
topTen

product  \
13443    Ultimate Antioxidant C Boost Serum   
449    20% Vitamin C + E Ferulic Acid Serum   
2218                      C-Firma Day Serum   
11015                    Vitamin C Infusion   
3552                  Superskin Moisturiser   
8659                                    TMR   
11806        Everlasting Anti-Aging Essence   
12500            Royalty Renewing Day Cream   
5266                    Alpha Beta Glow Pad   
8482                     Brightening Elixir   

                                      brand  \
13443                        Youth Corridor   
449                                Timeless   
2218                         Drunk Elephant   
11015                             Pulsaderm   
3552   Liz Earle Naturally Active Skin Care   
8659                                 Vivier   
11806                             Zephyrine   
12500                              Younique   
5266              Dr. Dennis Gross Skincare   
8482                           Goldfaden MD   

                                                 ingList  commonIng  
13443  [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...         11  
449    [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...          8  
2218   [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...          9  
11015  [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...         10  
3552   [Water, Vaccinium Macrocarpon (Cranberry) Seed...          4  
8659   [Ethoxydiglycol, Propylene Glycol, Water, Acet...          3  
11806  [Water, Disodium EDTA, Allantoin, Carbomer, Tr...          4  
12500  [Water, Cyclopentasiloxane, Glycerin, Polymeth...          3  
5266   [Water, Ethoxydiglycol, Dihydroxyacetone, Glyc...          5  
8482   [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...          8

## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

In [71]:
#%%time

recommended = get_RBO_recommendations(df, product, rbo_min)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/RBO.csv')

In [72]:
topTen

product  \
13443    Ultimate Antioxidant C Boost Serum   
449    20% Vitamin C + E Ferulic Acid Serum   
2218                      C-Firma Day Serum   
8482                     Brightening Elixir   
8097              C Perfect Vitamin C Serum   
11355           Vitamin C Superactive Serum   
11015                    Vitamin C Infusion   
8660                         Radiance Serum   
5266                    Alpha Beta Glow Pad   
4264    Sunless Tan Body Tanning Towelettes   

                                                 ingList  commonIng  
13443  [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...         11  
449    [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...          8  
2218   [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...          9  
8482   [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...          8  
8097   [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...          8  
11355  [Ethoxydiglycol, Water, Ascorbic Acid (Vitamin...          4  
11015  [Water, Ethoxydiglycol, Ascorbic Acid (Vitamin...         10  
8660   [Ethoxydiglycol, Propylene Glycol, Water, Asco...          4  
5266   [Water, Ethoxydiglycol, Dihydroxyacetone, Glyc...          5  
4264   [Water, Ethoxydiglycol, Dihydroxyacetone, Buty...          4